# フォルダ内にあるPDFファイルをJPEGに変換する

# 必要なパッケージのインストール

In [17]:
conda install -c conda-forge poppler pdf2image

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
#フォルダ内のファイルを全読み込み⇒変換
import glob
import os
from pathlib import Path
from pdf2image import convert_from_path

# PDFファイルのパス
pdf_path = Path("./ドキュメント/")

for path in glob.glob(os.path.join(pdf_path, '*.pdf')):
    
    #outputのファイルパス
    img_path=Path("./ドキュメント/PDFtoJPEG")

    #この1文で変換されたjpegファイルが、imageフォルダに作られます
    convert_from_path(path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem)



# DSTのCSVファイルを図面にプロットにする


In [9]:
#①DSTファイルを読み込んでプロット用に整形する

#必要なライブラリの読み込み
import numpy as np
import pylab
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
#from matplotlib.patches import Rectangle

#DSTのCSVファイルを読み込む
df = pd.read_csv('/home/yorito/ドキュメント/0110history.csv',engine="python",encoding='cp932',dtype={'PCI/PSC': str})

#RSRPの色分けカラムを追加
df.loc[df['RSRP/RSSI'] < -111,'color'] = 'red'
df.loc[df['RSRP/RSSI'] > -110,'color'] = 'yellow'
df.loc[df['RSRP/RSSI'] > -105,'color'] = 'blue'

#アノテーションで表示するラベルのカラムの設定
#df['label'] = df['PCI/PSC'] + ': '+df['RSRP/RSSI'].astype(str) + ' / ' + df['RSRQ'].astype(str) 
df['label'] = 'M' + df['mark_No'].astype(str)

df

,mark_No,date_time,RSRP/RSSI,RSRQ,PCI/PSC,EARFCN/UARFCN,RAT,color,label
0,2,2021/01/07 13:10:35,-94,-7,286,276,LTE,blue,M2
1,3,2021/01/07 13:12:13,-91,-6,286,276,LTE,blue,M3
2,4,2021/01/07 13:16:34,-89,-7,468,6100,LTE,blue,M4
3,5,2021/01/07 13:19:02,-83,-7,286,276,LTE,blue,M5
4,6,2021/01/07 13:20:48,-100,-9,286,276,LTE,blue,M6
5,7,2021/01/07 13:22:03,-98,-7,468,6100,LTE,blue,M7
6,8,2021/01/07 13:24:17,-96,-8,468,6100,LTE,blue,M8
7,9,2021/01/07 13:25:39,-87,-7,468,6100,LTE,blue,M9
8,10,2021/01/07 13:26:51,-80,-7,468,6100,LTE,blue,M10
9,11,2021/01/07 13:28:48,-93,-7,468,6100,LTE,blue,M11


In [18]:
#②図面をクリックした場所にDSTの結果をプロットする　inline でインライン表示
%matplotlib tk

#変数
global ind #プロット用マーク番号
global clickcount
global mark_point #プロット用マーク番号のdataframe
global JPEG_filePath


#画像をクリックした時座標を記録するメソッド
def onclick(event):
    global ind
    global mark_point
    global clickcount
    
    #デバッグ用
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    
    #クリックした座標をアサインする
    mark_point = mark_point.append({'mark_No':int(ind),'xdata':event.xdata,'ydata':event.ydata},ignore_index=True)
    
    #クリックした場所をリアルタイムに表示する
    plt.title("You clicked Mark_No="+str(df.iloc[ind,0]))
    ax.plot([event.xdata],[event.ydata],"o",color="red",alpha=1)
    ax.text(mark_point.iloc[clickcount,1],mark_point.iloc[clickcount,2],df.iloc[ind,8],bbox=dict(boxstyle='round,pad=0.2', fc=df.iloc[ind,7], alpha=0.9))
    ind +=1
    clickcount +=1
    print(ind)
    fig.canvas.draw()

#-----メインの処理-----
#フロア図のファイルパスを指定
JPEG_filePath = "/home/yorito/ドキュメント/PDFtoJPEG/0001-1.jpg"
#変数の初期化-マーク番号
ind = input('プロットする最初のマーク番号を入力：')
ind = int(ind)
#ind = 0
clickcount = 1

#クリックしたポイントの座標を格納するDataFrameを初期化
mark_point = pd.DataFramemark_point = pd.DataFrame(columns=['mark_No','xdata','ydata'])



#クリック用画像の表示
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
# 縦軸・横軸の目盛とラベルを非表示
ax.tick_params(left=False, bottom=False,labelleft=False,labelbottom=False) 

#ax.plot(x,y)
cid = fig.canvas.mpl_connect('button_press_event', onclick)

#背景の画像（フロア図）を指定する/home/yorito/ドキュメント/
im = Image.open(JPEG_filePath)
ax.imshow(im, alpha=1)

plt.show()

プロットする最初のマーク番号を入力： 6


# DataFrameをエクセルにエクスポートする

In [10]:
df.to_excel('/home/yorito/ドキュメント/pandas_to_excel.xlsx', sheet_name='new_sheet_name')
print("Export だん")

Export だん


In [13]:
mark_point

,mark_No,xdata,ydata
0,10.0,1150.898387,786.172581
1,10.0,1148.645161,808.704839
2,10.0,290.166129,808.704839
3,10.0,970.640323,536.064516
4,10.0,1137.379032,403.124194
5,10.0,1159.911290,324.261290
6,10.0,1135.125806,231.879032
